1. 安装组件

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install --upgrade transformers trl peft bitsandbytes accelerate datasets pandas modelscope swanlab

2. 加载数据

In [3]:
from modelscope.msdatasets import MsDataset
import json
import random
import os

# 设置随机种子
random.seed(42)

# 加载数据集
ds = MsDataset.load(
    'krisfu/delicate_medical_r1_data',
    subset_name='default',
    split='train',
    trust_remote_code=True  # 显式声明
)
data_list = list(ds)
random.shuffle(data_list)

# 划分训练集和验证集
split_idx = int(len(data_list) * 0.9)
train_data = data_list[:split_idx]
val_data = data_list[split_idx:]

# 创建 data 目录（如果不存在）
os.makedirs('data', exist_ok=True)

# 保存到 data 目录下的 jsonl 文件
with open('data/train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

with open('data/val.jsonl', 'w', encoding='utf-8') as f:
    for item in val_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"The dataset has been split successfully.")
print(f"Train Set Size: {len(train_data)}")
print(f"Val Set Size: {len(val_data)}")

2025-08-27 12:26:55,287 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from delicate_medical_r1_data. Please make sure that you can trust the external codes.
2025-08-27 12:26:55,943 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from krisfu/delicate_medical_r1_data. Please make sure that you can trust the external codes.
2025-08-27 12:26:55,944 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from krisfu/delicate_medical_r1_data. Please make sure that you can trust the external codes.
2025-08-27 12:26:55,944 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from krisfu/delicate_medical_r1_data. Please make sure that you can trust the external codes.


The dataset has been split successfully.
Train Set Size: 2166
Val Set Size: 241


3. 开始训练

In [9]:
# -*- coding: utf-8 -*-
"""
医学问答模型QLoRA高效微调脚本 (V3.0 终极新手注释版)

【本脚本的使命】
这份脚本是为所有对大模型微调感兴趣的新手朋友们量身打造的“保姆级”教程。
我们坚信,最好的学习是“在实践中学习”。因此,我们不仅提供了完整、可运行的代码,
更在每一个关键环节都附上了详尽的、口语化的“极致注释”。

我们的目标是：
1.  【知其然】: 您只需修改少数几个路径和参数,就能成功运行第一次微调。
2.  【知其所以然】: 通过阅读注释,您能理解每个参数的意义、每个函数的作用,以及它们对最终结果的影响。
3.  【赋予您调优的能力】: 在理解的基础上,您可以自信地调整超参数,针对您自己的任务进行优化。

【核心特性】
1.  [稳定可靠] 彻底解决了新手常遇到的"torch.xpu"环境错误。
2.  [功能完整] 包含了QLoRA、Flash Attention 2、自动化实验命名等所有先进特性。
3.  [极致注释] 可能是您能找到的、注释最详尽的中文微调脚本。
4.  [实验可追溯] 深度集成了SwanLab,让您的每一次尝试都有迹可循。
"""

# =====================================================================================
# 【第零步：环境准备】- 在代码运行前,让计算机做好准备
# =====================================================================================
import os

# 【！！！核心重点！！！】环境变量配置 - 必须在所有import PyTorch等库之前执行
# 这部分代码就像在演出开始前布置舞台,确保所有演员（库）都能在正确的环境下工作。

# 【GPU设备选择】
#   - 是什么：告诉程序我们想使用哪一块NVIDIA显卡（GPU）。计算机中可能有多块显卡,编号从0开始。
#   - 为什么是"0"：通常,我们使用性能最好或者当前空闲的第一块显卡。
#   - 改了会怎样：如果您有多块显卡,可以设置为"0", "1", "2"等,或者"0,1"来同时使用两块（需要修改代码以支持多卡训练）。
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 【！！！核心修复！！！】解决 'torch.xpu' 错误的“魔法开关”
#   - 是什么：这是一组环境变量,用来阻止程序去探测和使用Intel的XPU（一种非NVIDIA的处理器）。
#   - 为什么需要：很多开源库（如transformers, accelerate）默认会检查所有类型的硬件,
#              但在一个只有NVIDIA GPU的环境下,这种检查可能会因为缺少Intel驱动而报错。
#   - 设置了会怎样：这两行代码像“告示牌”,告诉所有库：“此路不通,请直接走NVIDIA CUDA通道”,从而避免了错误。
os.environ["DISABLE_IPEX"] = "1"
os.environ["DEEPSPEED_XPU_ENABLE"] = "0"

# 【内存分配优化】
#   - 是什么：告诉PyTorch如何管理GPU内存。
#   - 为什么设置：可以减少内存碎片,有时能避免"CUDA out of memory"（显存不足）的错误,让程序能更灵活地使用显存。
#   - 类比：就像一个聪明的行李打包员,它会尽量把大小不一的行李（数据）紧凑地塞进后备箱（显存）,而不是随意乱放导致空间浪费。
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


# =====================================================================================
# 【第一步：导入工具包】 - 准备好我们需要的“瑞士军刀”
# =====================================================================================
# 导入我们完成任务所需要的所有Python库

import json                 # 用于读写JSON格式的数据文件。
import pandas as pd         # 一个强大的数据分析工具,能方便地读取和处理我们的数据集。
import torch                # PyTorch,深度学习的核心框架,所有模型训练和计算都基于它。
from datasets import Dataset # Hugging Face的datasets库,用于高效地加载和处理大规模文本数据。
from modelscope import snapshot_download # ModelScope的工具,用于从国内镜像源下载模型,速度更快。

# 【Transformers库】 - 这是与大语言模型交互的核心库
from transformers import (
    AutoTokenizer,          # 自动分词器,能将文本转换成模型能理解的数字（Token）。
    AutoModelForCausalLM,   # 自动模型加载器,用于加载像Qwen这样的生成式大模型。
    TrainingArguments,      # 训练参数配置器,一个包含了所有训练选项的“控制面板”。
    Trainer,                # 训练器,封装了复杂的训练循环,让我们用几行代码就能开始训练。
    DataCollatorForSeq2Seq, # 数据整理器,能将多条长短不一的数据智能地打包成一个批次（Batch）。
    BitsAndBytesConfig      # QLoRA的量化配置器,用来设置如何将模型从32位压缩到4位。
)
import swanlab              # 一个实验跟踪工具,能像“黑匣子”一样记录我们训练过程中的所有指标。

# 【PEFT库】 - 实现高效微调的“魔法”所在
# Parameter-Efficient Fine-Tuning (参数高效微调)
from peft import (
    LoraConfig,             # LoRA的配置器,用来定义“小模型”（适配器）的结构。
    get_peft_model,         # 将LoRA适配器“安装”到大模型上的工具函数。
    prepare_model_for_kbit_training, # 为k-bit（如4-bit）量化后的模型做训练前的准备工作。
    PeftModel               # 用于在推理时加载训练好的LoRA适配器。
)


def check_environment():
    """环境检查函数,确保CUDA（NVIDIA GPU的计算平台）可用。"""
    if not torch.cuda.is_available():
        raise RuntimeError("❌ CUDA不可用！此脚本需要NVIDIA GPU来运行QLoRA微调。")
    print(f"✅ CUDA环境检查通过！")
    print(f"📊 可用GPU数量: {torch.cuda.device_count()}")
    print(f"🔧 当前CUDA设备: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU总显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    # 清空一下显存,以防之前有其他程序占用了显存。
    torch.cuda.empty_cache()

# =====================================================================================
# 【第二步：全局配置】 - 搭建我们微调实验的“控制中心”
# 在这里,您可以像导演一样,设定整个微调剧本的核心要素。
# =====================================================================================

# --- 路径配置 (Path Configuration) ---
# 良好的路径管理是项目成功的开始,让所有文件各得其所。
BASE_DIR = "."  # 项目根目录,"."代表当前文件夹
DATA_DIR = os.path.join(BASE_DIR, "data")                       # 存放原始数据的文件夹
FORMATTED_DATA_DIR = os.path.join(BASE_DIR, "data_formatted")   # 存放处理后数据的文件夹
MODEL_CACHE_DIR = os.path.join(BASE_DIR, "model_cache")         # 存放下载的基础模型的文件夹,避免重复下载
OUTPUT_DIR = os.path.join(BASE_DIR, "output")                   # 存放所有训练输出的根文件夹
CHECKPOINTS_DIR = os.path.join(OUTPUT_DIR, "checkpoints")       # 存放训练过程中的检查点（临时存档）
FINAL_ADAPTER_DIR = os.path.join(OUTPUT_DIR, "final_adapter")   # 存放最终训练好的、可供分享的LoRA适配器

# --- 模型与提示词配置 (Model and Prompt Configuration) ---
# 【模型ID MODEL_ID】
#   - 是什么：您选择的基础大模型的唯一标识符。可以来自Hugging Face Hub或ModelScope。
#   - 为什么是"Qwen/Qwen3-1.7B"：这是一个优秀的中英文开源模型,参数量较小（17亿）,适合在消费级显卡上进行微调。
#   - 改了会怎样：您可以换成任何您想微调的模型ID,比如"Qwen/Qwen3-7B", "meta-llama/Llama-3-8b"等,但请注意模型越大,显存消耗也越大。
MODEL_ID = "Qwen/Qwen3-1.7B"

# 【！！！核心重点！！！】系统提示词 (SYSTEM_PROMPT)
#   - 是什么：给模型设定的“人设”或“行为准则”,告诉它应该扮演什么角色,遵循哪些规则。
#   - 为什么重要：这里的PROMPT必须与您未来进行推理、评估、部署时使用的SYSTEM_PROMPT完全一致！
#   - 类比：这相当于在培训员工（微调）和考核员工（推理）时,使用的是同一份岗位说明书。如果说明书不一致,员工（模型）就会感到困惑,表现自然会打折扣。
SYSTEM_PROMPT = """你是一个专业、严谨的AI医学助手。你的任务是根据用户提出的问题,提供准确、易懂且具有安全提示的健康信息。请记住,你的回答不能替代执-业医师的诊断,必须在回答结尾处声明这一点。"""

# 【最大序列长度 MAX_LENGTH】
#   - 是什么：一条训练数据（问题+答案）被转换成数字（Token）后,允许的最大长度。
#   - 为什么是2048：对于大多数问答任务,这是一个比较均衡的值。它能容纳足够长的上下文,同时显存消耗也在可接受范围内。
#   - 改了会怎样：
#     - 增加（如4096）：可以处理更长的问答对,但会显著增加显存消耗（显存占用与长度的平方成正比！）。
#     - 减少（如1024）：节省显存,训练更快,但如果您的数据普遍较长,可能会因截断而丢失重要信息,影响模型学习效果。

MAX_LENGTH = 2048

# --- 【！！！核心重点！！！】训练超参数 (Hyperparameters) ---
# 这部分是微调的“灵魂”,是您可以调节来提升模型效果的关键“旋钮”。

# 【学习率 Learning Rate】
#   - 是什么：可以理解为模型在学习过程中“更新知识”的步伐大小。
#   - 为什么是1e-4：对于只训练“小本子”（LoRA适配器）的QLoRA方法,我们可以让它学得快一点。1e-4是一个经过大量实验验证的、对QLoRA来说效果很好的起始值。相比之下,全量微调（训练整个模型）通常使用更小的值（如2e-5）,因为步伐太大容易“跑偏”。
#   - 改了会怎样：
#     - 太高（如1e-3）：像是一个学生看书一目十行,可能什么都没学进去,导致训练不稳定,损失（loss）不下降甚至上升。
#     - 太低（如1e-5）：像是一个学生逐字逐句地钻牛角尖,学习速度会非常慢,可能训练很久效果都提升不明显。
LEARNING_RATE = 1e-4

# 【单设备批量大小 PER_DEVICE_TRAIN_BATCH_SIZE】
#   - 是什么：一次性喂给GPU多少条数据进行学习。
#   - 为什么是8：这是一个在24GB显存下比较均衡的值。值越小,单次计算的显存占用越低。
#   - 改了会怎样：
#     - 增大：训练速度会加快,但显存占用会急剧增加。如果设置过大,会直接导致“CUDA out of memory”错误。
#     - 减小：能有效降低显存占用,是解决显存不足问题的首选方案。但训练速度会变慢。
PER_DEVICE_TRAIN_BATCH_SIZE = 8

# 【梯度累积步数 GRADIENT_ACCUMULATION_STEPS】
#   - 是什么：一种“用时间换空间”的技巧,在不增加显存的情况下,实现等效于更大Batch Size的训练效果。
#   - 为什么是8：配合上面的`BATCH_SIZE = 8`,我们的“有效批量大小”(Effective Batch Size) = 8 * 8 = 64。这通常是一个能让模型稳定学习的批量大小。
#   - 工作原理：程序会先计算8次小批量（每次8条数据）的梯度,但先不更新模型,而是把这8次的梯度“累积”起来,最后用这个累积的梯度完成一次模型更新。这在数学效果上等同于一次性处理64条数据。
#   - 改了会怎样：它和`BATCH_SIZE`是跷跷板关系。当显存不足时,您可以降低`BATCH_SIZE`（比如降到4）,同时提高此值（比如提高到16）,以保持“有效批量大小”不变（4 * 16 = 64）,这样训练效果基本不受影响。
GRADIENT_ACCUMULATION_STEPS = 8

# 【训练轮数 NUM_TRAIN_EPOCHS】
#   - 是什么：将整个训练数据集从头到尾完整地学习多少遍。
#   - 为什么是2：对于微调任务,尤其是在高质量的数据集上,通常1-3个epoch就足够了。这可以在较短时间内看到明显效果,同时避免模型“死记硬背”。
#   - 改了会怎样：
#     - 太少（如1）：可能导致模型“没学透”,效果提升不明显。
#     - 太多（如5+）：可能发生“过拟合”。即模型只会死记硬背训练数据,丧失了在新问题上的推理和泛化能力,就像一个只会做练习册但一到考场就蒙圈的学生。
NUM_TRAIN_EPOCHS = 2

# --- 训练过程控制参数 ---
EVAL_STEPS = 100        # 每训练100步,就用验证集评估一次模型,像是一次模拟考,帮助我们监控学习进度。
SAVE_STEPS = 400        # 每训练400步,就保存一次模型检查点,防止电脑突然断电导致训练成果白费。
LOGGING_STEPS = 10      # 每训练10步,就在控制台打印一次当前的训练损失（loss）,让我们能实时看到学习状态。

# --- 【！！！核心重点！！！】LoRA配置 (LoRA Configuration) ---
# LoRA是高效微调的魔法。可以把它想象成我们不改变一个博学的教授（基础大模型）,
# 而是给他配一个“专业领域笔记本”（LoRA适配器）,我们只训练这个笔记本,让教授学会新知识。

# 【LoRA秩 R - 笔记本的“厚度”】
#   - 是什么：LoRA适配器矩阵的秩,决定了这个“笔记本”能记录多少新知识。
#   - 为什么是32：R值通常在8, 16, 32, 64中选择。32是一个很好的平衡点,在性能和参数量之间取得了很好的平衡。
#   - 改了会怎样：
#     - 增大（如64, 128）：笔记本更“厚”,可训练的参数更多,理论上拟合能力更强,但显存占用也更大,且过大也可能导致过拟合。
#     - 减小（如16, 8）：笔记本更“薄”,训练更快,显存更省,但对于复杂任务可能“记不住”那么多知识,效果稍差。
LORA_R = 32

# 【LoRA缩放因子 Alpha - 笔记本内容的“重要性”】
#   - 是什么：一个缩放参数,用来调整LoRA适配器（笔记本）对最终结果的影响权重。
#   - 为什么是64：一个广为流传且效果很好的经验法则是将alpha设置为r的两倍 (`alpha = 2 * r`)。
#   - 改了会怎样：它与学习率有类似的作用,调整它会影响LoRA模块的权重大小。保持`2*r`的比例通常是最佳实践,新手可以不用修改。
LORA_ALPHA = 64

# 【LoRA Dropout - 防止“死记硬背”的遗忘机制】
#   - 是什么：在训练时,随机地“丢弃”一部分LoRA参数,不让它们参与当次计算。
#   - 为什么是0.05：这是一个常用的值。它可以增强模型的泛化能力。
#   - 类比：就像是故意让学生在复习时随机忘掉5%的知识点,强迫他去理解知识本身,而不是死记硬背答案。
LORA_DROPOUT = 0.05

# 【目标模块 TARGET_MODULES - 笔记应该“贴”在哪里】
#   - 是什么：告诉LoRA应该在模型的哪些部分“加装”适配器。
#   - 为什么是这几个：`q_proj`, `k_proj`, `v_proj`, `o_proj`是Transformer模型中“注意力机制”的四个核心组件。注意力机制决定了模型如何理解文本的上下文关系,对它们进行微调通常效果最显著。
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj"]


# =====================================================================================
# 【第三步：动态命名与实验跟踪】 - 让我们的实验井井有条
# =====================================================================================

# --- 动态命名配置 (Dynamic Naming Configuration) ---
# 【原理】从"Qwen/Qwen3-1.7B"这样的模型ID中,自动提取出"Qwen3-1.7B"作为基础名称。
# 【优势】当您更换MODEL_ID时,所有相关的项目名、运行名和文件夹名都会自动更新,无需手动修改,
#         这使得实验管理更加清晰、自动化,避免了手动改名带来的疏忽和错误。
model_short_name = MODEL_ID.split('/')[-1]

# --- SwanLab实验跟踪配置 ---
# 【动态项目名】项目名现在由模型基础名和任务类型（medical-qlora）动态构成。
#   - 这使得在SwanLab中,所有基于同一个基础模型的实验都会被自动归类到同一个项目下。
os.environ["SWANLAB_PROJECT"] = f"{model_short_name}-medical-qlora"

EFFECTIVE_BATCH_SIZE = PER_DEVICE_TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS

# 【动态运行名】为每次训练生成一个唯一的、包含核心超参数的名称,便于在SwanLab中对比不同实验。
#   - 例如："Qwen3-1.7B-qlora-lr1e-4-bs64-r32"
#   - 看到这个名字,您就能立刻知道这次实验是用哪个模型,学习率、批量、LoRA秩各是多少。
RUN_NAME = f"{model_short_name}-qlora-lr{LEARNING_RATE}-bs{EFFECTIVE_BATCH_SIZE}-r{LORA_R}"

# 将所有超参数记录到SwanLab中,方便回溯和分析。
swanlab.config.update({
    "model": MODEL_ID,
    "model_short_name": model_short_name,
    "system_prompt": SYSTEM_PROMPT,
    "max_length": MAX_LENGTH,
    "learning_rate": LEARNING_RATE,
    "epochs": NUM_TRAIN_EPOCHS,
    "effective_batch_size": EFFECTIVE_BATCH_SIZE,
    "lora_r": LORA_R,
    "lora_alpha": LORA_ALPHA,
    "lora_dropout": LORA_DROPOUT,
})

# =====================================================================================
# 【第四步：工具函数】 - 定义一些方便我们重复使用的“小工具”
# 这部分代码通常不需要修改。
# =====================================================================================

def setup_directories():
    """创建项目所需的所有目录结构,如果不存在的话。"""
    directories = [DATA_DIR, FORMATTED_DATA_DIR, MODEL_CACHE_DIR, CHECKPOINTS_DIR, FINAL_ADAPTER_DIR]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
    print("✅ 所有项目目录已准备就绪")

def dataset_jsonl_transfer(origin_path, new_path):
    """将我们自定义的原始数据格式,转换为更通用的{"input": "...", "output": "..."}格式。"""
    messages = []
    try:
        with open(origin_path, "r", encoding="utf-8") as file:
            for line_num, line in enumerate(file, 1):
                try:
                    data = json.loads(line.strip())
                    if "question" not in data or "think" not in data or "answer" not in data:
                        print(f"⚠️ 警告：第{line_num}行缺少必要字段,跳过")
                        continue
                    # 将问题作为输入
                    input_text = data["question"]
                    # 将思考过程和答案合并作为输出
                    output_text = f'<|FunctionCallBegin|>{data["think"]}<|FunctionCallEnd|>\n{data["answer"]}'
                    messages.append({"input": input_text, "output": output_text})
                except json.JSONDecodeError:
                    print(f"⚠️ 警告：第{line_num}行JSON格式错误,跳过")
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ 找不到数据文件: {origin_path}")
    
    with open(new_path, "w", encoding="utf-8") as file:
        for message in messages:
            file.write(json.dumps(message, ensure_ascii=False) + "\n")
    print(f"✅ 数据转换完成: {len(messages)} 条有效数据")


def process_func(example, tokenizer):
    """
    【！！！核心重点！！！】数据处理函数 - 将一条文本数据转换为模型能“吃”的格式
    这是整个微调流程中最关键的函数之一,它定义了模型如何学习。
    """
    
    # 【构建符合Qwen聊天模板的输入格式】
    #   - 模型就像一个演员,需要清晰的剧本才能演好戏。这个模板就是剧本。
    #   - `<|im_start|>` 和 `<|im_end|>` 是Qwen模型约定的特殊标记,用来区分不同角色的对话。
    #   - `system`: 系统指令,我们在这里放入之前定义的SYSTEM_PROMPT。
    #   - `user`: 用户的提问,我们放入数据中的'input'字段。
    #   - `assistant`: AI助手的回答部分,我们在这里留一个开头,让模型接着生成。
    instruction_text = (
        f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n"
        f"<|im_start|>user\n{example['input']}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    
    # 使用分词器将文本转换为数字ID。`add_special_tokens=False`因为我们已经手动添加了模板。
    instruction = tokenizer(instruction_text, add_special_tokens=False)
    response = tokenizer(example['output'], add_special_tokens=False)

    # 【拼接完整的输入序列】
    # 完整的输入 = 系统指令部分 + 用户问题部分 + 助手回答部分 + 结束标记
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    
    # 【！！！核心重点！！！】设计标签(labels) - 告诉模型应该学习什么
    #   - 这是监督学习的核心：为模型的学习“划重点”。
    #   - 我们希望模型学会“在看到用户问题后,生成我们提供的答案”,而不是学会“复述用户的问题”。
    #   - 因此,我们将系统指令和用户问题部分的标签设置为-100。
    #   - 在PyTorch中,-100是一个特殊的忽略索引,意味着在计算损失（loss,即模型的回答与标准答案的差距）时,这些部分不参与计算。
    #   - 只有助手回答部分的标签是真实的Token ID,模型会努力让自己的预测接近这些ID。
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    
    # 【长度截断】
    # 如果拼接后的序列超过了我们设定的最大长度MAX_LENGTH,就进行截断,防止显存溢出。
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    # attention_mask（注意力掩码）全为1,表示模型需要关注序列中的每一个token。
    return {"input_ids": input_ids, "attention_mask": [1] * len(input_ids), "labels": labels}

def predict(messages, model, tokenizer):
    """使用微调后的模型进行一次对话预测。"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    # `torch.no_grad()`表示在推理时我们不需要计算梯度,这可以节省大量显存并加快速度。
    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=512,        # 回答的最大长度
            temperature=0.3,           # 温度较低,回答更具确定性,适合严谨的医学领域
            repetition_penalty=1.1,    # 轻微惩罚重复,避免模型说车轱辘话
            do_sample=True,            # 启用采样,让回答更自然
            top_p=0.8,                 # 从概率最高的80%词汇中采样
            pad_token_id=tokenizer.eos_token_id
        )
        
    # 从生成结果中剔除输入部分,只保留新生成的回答。
    response_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    response = tokenizer.decode(response_ids, skip_special_tokens=True)
    return response

# =====================================================================================
# 【第五步：主执行流程】 - 将所有零件组装起来,启动微调！
# =====================================================================================
def main():
    """主训练流程函数"""
    
    # --- 步骤1: 环境与路径初始化 ---
    print("🚀 开始QLoRA医学问答微调流程...")
    check_environment()
    setup_directories()

    # --- 步骤2: 加载分词器和基础模型 ---
    print(f"\n📥 正在从'{MODEL_ID}'加载基础模型和分词器...")
    model_dir = snapshot_download(MODEL_ID, cache_dir=MODEL_CACHE_DIR, revision="master")
    tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True)
    # 为我们自定义的特殊标记<|FunctionCall...|>扩展词汇表
    tokenizer.add_special_tokens({'additional_special_tokens': ['<|FunctionCallBegin|>', '<|FunctionCallEnd|>']})
    # 如果模型没有默认的pad_token,就用eos_token（结束标记）代替,这是常见做法。
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("✅ 分词器加载并扩展完成")

    # --- 步骤3: 配置QLoRA量化 ---
    print("\n⚙️ 配置4-bit量化参数(QLoRA)...")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,                      # 【核心】启用4-bit量化,将模型权重从32位压缩到4位。
        bnb_4bit_quant_type="nf4",              # 使用NF4（Normal Float 4）量化类型,这是一种专为神经网络优化的格式。
        bnb_4bit_compute_dtype=torch.bfloat16,  # 在计算时,临时将权重恢复到bfloat16精度,以保证计算的准确性。
        bnb_4bit_use_double_quant=True,         # 启用双重量化,进一步节省显存。
    )

    # --- 步骤4: 加载量化模型并应用LoRA ---
    print("\n🔧 加载量化模型并应用LoRA...")
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_dir,
            device_map="auto",                  # 自动将模型分层加载到可用的GPU和CPU上,以最大化利用资源。
            torch_dtype=torch.bfloat16,         # 指定计算时使用的数据类型。
            quantization_config=quantization_config, # 应用我们上面定义的4-bit量化配置。
            attn_implementation="flash_attention_2" # 【性能优化】使用Flash Attention 2,能大幅提升训练速度并节省显存。
        )
        print("✅ 量化模型加载完成 (Flash Attention 2 已启用)")
    except Exception as e:
        print(f"⚠️ Flash Attention 2加载失败,将使用标准注意力机制: {e}")
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, device_map="auto", torch_dtype=torch.bfloat16,
            quantization_config=quantization_config
        )

    # 调整模型的词嵌入层大小,以适应我们新增的特殊Token。
    model.resize_token_embeddings(len(tokenizer))
    # 为量化后的模型做一些训练前的准备工作。
    model = prepare_model_for_kbit_training(model)
    # 定义LoRA配置。
    lora_config = LoraConfig(
        r=LORA_R, lora_alpha=LORA_ALPHA, target_modules=TARGET_MODULES,
        lora_dropout=LORA_DROPOUT, bias="none", task_type="CAUSAL_LM"
    )
    # 将LoRA配置“安装”到模型上。
    model = get_peft_model(model, lora_config)
    # 打印出可训练参数的数量和比例,您会发现它非常小！这就是QLoRA的魅力。
    model.print_trainable_parameters()
    print("✅ LoRA适配器已成功应用")

    # --- 步骤5: 数据集准备与预处理 ---
    print("\n📊 准备训练数据...")
    train_original_path = os.path.join(DATA_DIR, "train.jsonl")
    val_original_path = os.path.join(DATA_DIR, "val.jsonl")
    train_formatted_path = os.path.join(FORMATTED_DATA_DIR, "train_formatted.jsonl")
    val_formatted_path = os.path.join(FORMATTED_DATA_DIR, "val_formatted.jsonl")
    # 如果格式化数据不存在,则进行转换
    if not os.path.exists(train_formatted_path): dataset_jsonl_transfer(train_original_path, train_formatted_path)
    if not os.path.exists(val_formatted_path): dataset_jsonl_transfer(val_original_path, val_formatted_path)
    # 加载数据集
    train_dataset = Dataset.from_pandas(pd.read_json(train_formatted_path, lines=True))
    eval_dataset = Dataset.from_pandas(pd.read_json(val_formatted_path, lines=True))
    # 使用.map()方法对数据集中的每一条数据应用我们的process_func函数
    tokenized_train_dataset = train_dataset.map(lambda x: process_func(x, tokenizer), remove_columns=train_dataset.column_names)
    tokenized_eval_dataset = eval_dataset.map(lambda x: process_func(x, tokenizer), remove_columns=eval_dataset.column_names)
    print("✅ 数据预处理完成")

    # --- 步骤6: 配置训练参数 ---
    print("\n⚙️ 配置训练参数...")
    training_args = TrainingArguments(
        output_dir=os.path.join(CHECKPOINTS_DIR, RUN_NAME), # 所有输出（检查点、日志等）都将保存在这个动态生成的文件夹中
        per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        learning_rate=LEARNING_RATE,
        num_train_epochs=NUM_TRAIN_EPOCHS,
        # 【优化器】使用paged_adamw_8bit,这是QLoRA推荐的、能进一步节省显存的优化器。
        optim="paged_adamw_8bit",
        # 【学习率调度器】使用余弦退火调度器,可以在训练开始时缓慢增加学习率（预热）,然后在训练过程中平滑地降低学习率。
        lr_scheduler_type="cosine",
        warmup_ratio=0.1, # 前10%的训练步数用于预热。
        # 【评估与保存】
        eval_strategy="steps",
        eval_steps=EVAL_STEPS,
        save_steps=SAVE_STEPS,
        save_total_limit=3, # 最多只保留最近的3个检查点,避免占用过多磁盘空间。
        # 【日志与报告】
        logging_steps=LOGGING_STEPS,
        report_to="swanlab", # 将训练指标上报给SwanLab。
        run_name=RUN_NAME,   # 在SwanLab中显示的实验名称。
        # 【模型加载策略】
        load_best_model_at_end=True,        # 训练结束后,自动加载在验证集上表现最好的那个检查点。
        metric_for_best_model="eval_loss",  # 判断“最佳”的标准是验证集损失（eval_loss）最低。
        greater_is_better=False,            # 损失（loss）是越小越好,所以设置为False。
        # 【精度与性能】
        bf16=True,                          # 使用bfloat16混合精度训练,可以提速并节省显存,且在现代GPU上比fp16更稳定。
        gradient_checkpointing=True,        # 【！！！核心显存优化！！！】另一个“用时间换空间”的关键技术。它在反向传播时重新计算中间激活值,而不是全部存储下来,能节省大量显存。
    )
    
    # --- 步骤7: 初始化训练器 ---
    print("\n🏃‍♂️ 初始化训练器...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        # 数据整理器,负责将数据样本智能地打包成批次
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    )
    print("✅ 训练器初始化完成")

    # --- 步骤8: 开始训练 ---
    print("\n🚀 开始模型训练...")
    print("="*80)
    print(f"  🦢 SwanLab 实验: '{RUN_NAME}'")
    print(f"  📂 项目名: '{os.environ['SWANLAB_PROJECT']}'")
    print(f"  🎯 模型: {MODEL_ID}, 🔄 轮数: {NUM_TRAIN_EPOCHS}, 📦 有效批量: {EFFECTIVE_BATCH_SIZE}")
    print("="*80)
    # 这行代码会启动整个训练过程,并自动处理所有循环、评估、保存等繁琐工作。
    trainer.train()
    print("🎉 训练完成!")

    # --- 步骤9: 保存最终模型 ---
    print("\n💾 保存最终的LoRA适配器...")
    # 我们只保存轻量级的LoRA适配器（“笔记本”）,而不是整个大模型。
    final_model_path = os.path.join(FINAL_ADAPTER_DIR, RUN_NAME)
    model.save_pretrained(final_model_path)
    tokenizer.save_pretrained(final_model_path) # 同时保存分词器,确保推理时能正确加载。
    print(f"✅ 最终适配器已保存到: {final_model_path}")

    # --- 步骤10: 模型测试 ---
    print("\n🧪 进行模型测试...")
    test_samples = eval_dataset.select(range(3)) # 从验证集中选几条数据进行测试
    test_results = []
    for sample in test_samples:
        # 我们需要将数字ID解码回文本才能进行测试
        question = tokenizer.decode([token for token in sample['input_ids'] if token != tokenizer.pad_token_id], skip_special_tokens=True)
        # 从Qwen模板中提取出真正的用户问题
        user_question = question.split("<|im_start|>user\n")[1].split("<|im_end|>")[0]
        
        messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_question}]
        response = predict(messages, model, tokenizer)
        
        # 将标准答案也解码出来
        ground_truth = tokenizer.decode([token for token in sample['labels'] if token != -100], skip_special_tokens=True)

        result_text = (
            f"🔍 **测试样本**:\n\n"
            f"**❓ 问题**: {user_question}\n\n"
            f"**🤖 模型回答**: {response}\n\n"
            f"**✅ 标准答案**: {ground_truth}\n"
        )
        test_results.append(swanlab.Text(result_text, caption="测试样本"))
        print(result_text + "-"*50 + "\n")
        
    if test_results:
        swanlab.log({"测试样例": test_results})

    # --- 步骤11: 训练总结 ---
    print("\n📋 训练总结与使用指导:")
    print("="*60)
    print(f"✅ 最终适配器已保存至: {final_model_path}")
    print("\n📝 模型使用指导:")
    print(f"1. 加载基础模型: `AutoModelForCausalLM.from_pretrained('{MODEL_ID}')`")
    print(f"2. 加载LoRA适配器: `PeftModel.from_pretrained(base_model, '{final_model_path}')`")
    print("3. 使用相同的SYSTEM_PROMPT和聊天模板进行推理。")
    print("="*60)

    swanlab.finish()
    print("\n🎉 QLoRA医学问答微调流程全部完成!")

# =====================================================================================
# 【第六步：程序入口】 - 整个脚本从这里开始执行
# =====================================================================================
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        # 允许用户通过 Ctrl+C 手动中断训练
        print("\n⏹️ 训练被用户中断")
    except Exception as e:
        # 捕获其他所有可能的错误,并打印详细信息,方便排查问题
        print(f"\n❌ 程序执行失败: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # 无论程序是成功结束还是中途失败,都确保关闭SwanLab的日志记录
        if 'swanlab' in globals():
            swanlab.finish()
        print("🔚 程序结束")

🚀 开始QLoRA医学问答微调流程...
✅ CUDA环境检查通过！
📊 可用GPU数量: 1
🔧 当前CUDA设备: NVIDIA A10
💾 GPU总显存: 23.7 GB
✅ 所有项目目录已准备就绪

📥 正在从'Qwen/Qwen3-1.7B'加载基础模型和分词器...


2025-08-27 15:16:50,047 - modelscope - INFO - Target directory already exists, skipping creation.


✅ 分词器加载并扩展完成

⚙️ 配置4-bit量化参数(QLoRA)...

🔧 加载量化模型并应用LoRA...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 量化模型加载完成 (Flash Attention 2 已启用)
trainable params: 12,845,056 || all params: 1,732,877,312 || trainable%: 0.7413
✅ LoRA适配器已成功应用

📊 准备训练数据...


Map:   0%|          | 0/2166 [00:00<?, ? examples/s]

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

✅ 数据预处理完成

⚙️ 配置训练参数...

🏃‍♂️ 初始化训练器...

❌ 程序执行失败: module 'torch.xpu' has no attribute 'get_arch_list'


Traceback (most recent call last):
  File "/tmp/ipykernel_524/2761892960.py", line 530, in <module>
    main()
  File "/tmp/ipykernel_524/2761892960.py", line 455, in main
    trainer = Trainer(
              ^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/transformers/utils/deprecation.py", line 172, in wrapped_func
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/transformers/trainer.py", line 631, in __init__
    unwrapped_model = self.accelerator.unwrap_model(model)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/accelerate/accelerator.py", line 3128, in unwrap_model
    return extract_model_from_parallel(model, keep_fp32_wrapper, keep_torch_compile)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/accelerate/utils/other.py", line 250, in extract_model_from_parallel
  

AttributeError: module 'swanlab' has no attribute 'is_running'